In [34]:
from ebooklib import epub
import os
from multiprocessing import Pool
import time
from bs4 import BeautifulSoup
import urllib2
import string

## deals with lit, mobi, azw3


In [154]:
d="/media/some/sophie-hhd/books/rtf/"
ls=!find $d -type f -iname "*.epub"  #-o -iname "*.chm" -o -iname "*.lit" -o -iname "*.azw3"
len(ls)

150

In [151]:
f=ls[1]
meta=!exiftool "$f"

In [152]:
metad={m.split(" : ")[0].strip():m.split(" : ")[1].strip() for m in meta }
metad

{'Creator': 'Unknown',
 'Directory': '/media/some/sophie-hhd/books/rtf',
 'ExifTool Version Number': '10.00',
 'File Access Date/Time': '2016:05:28 20:39:14+01:00',
 'File Inode Change Date/Time': '2016:05:28 20:39:14+01:00',
 'File Modification Date/Time': '2016:05:28 20:39:14+01:00',
 'File Name': '03 Beverly Barton - The Outcast 03.epub',
 'File Permissions': 'rwxrwxrwx',
 'File Size': '214 kB',
 'File Type': 'EPUB',
 'File Type Extension': 'epub',
 'Guide Reference Href': 'titlepage.xhtml',
 'Guide Reference Title': 'Title Page',
 'Guide Reference Type': 'cover',
 'Identifier': 'f5c237ce-3fbc-49e4-8bbe-788fde320046',
 'Identifier Id': 'uuid_id',
 'Identifier Scheme': 'uuid',
 'Language': 'en',
 'MIME Type': 'application/epub+zip',
 'Manifest Item Href': 'toc.ncx',
 'Manifest Item Id': 'ncx',
 'Manifest Item Media-type': 'application/x-dtbncx+xml',
 'Meta': '2016:05:28 20:39:14Z',
 'Meta Content': 'cover',
 'Meta Name': 'cover',
 'Meta Property': 'dcterms:modified',
 'Spine Itemref 

In [70]:
print metad['Creator']+"-"+metad['Title']+".epub"
print metad["File Name"]

Jeffery Deaver-The Skin Collector.epub
Jeffery Deaver [Lincoln Rhyme 11.0] - The Skin Collector.epub


In [127]:
key='http://purl.org/dc/elements/1.1/'
book=epub.read_epub(f)


In [128]:
metadata=book.metadata[key]

In [145]:
fn=os.path.basename(f)
m=fn[:-5].split("-")
series=extract(metadata["creator"][0][0],m[0] )
if len(series)>4:
    if "series" not in book.metadata[key].keys():
        book.metadata[key]['series']=[(series,{})]
    else:
        book.metadata[key]['series'].append((series,{}))
series=extract(metadata["creator"][0][0],m[1] )
if len(series)>4:
    if "series" not in book.metadata[key].keys():
        book.metadata[key]['series']=[(series,{})]
    else:
        book.metadata[key]['series'].append((series,{}))
series=extract(metadata["title"][0][0],m[0] )
if len(series)>4:
    if "series" not in book.metadata[key].keys():
        book.metadata[key]['series']=[(series,{})]
    else:
        book.metadata[key]['series'].append((series,{}))
series=extract(metadata["title"][0][0],m[1] )
if len(series)>4:
    if "series" not in book.metadata[key].keys():
        book.metadata[key]['series']=[(series,{})]
    else:
        book.metadata[key]['series'].append((series,{}))
if book.metadata[key]['title'][0][0].count("(")>0:
    tit=book.metadata[key]['title'][0][0].split("(")
    if len(' '.join(tit[0].split()))>4:
        title=' '.join(tit[0].split())
        series=tit[1].split(")")[0]
        #print title,"\n", series
    if len(series)>4:
        if "series" not in book.metadata[key].keys():
            book.metadata[key]['series']=[(series,{})]
        else:
            book.metadata[key]['series'].append((series,{}))
if book.metadata[key]['title'][0][0].count("[")>0:
    tit=book.metadata[key]['title'][0][0].split("[")
    if len(' '.join(tit[0].split()))>4:
        title=' '.join(tit[0].split())
        series=tit[1].split("]")[0]
        #print title,"\n", series
    if len(series)>4:
        if "series" not in book.metadata[key].keys():
            book.metadata[key]['series']=[(series,{})]
        else:
            book.metadata[key]['series'].append((series,{}))
if book.metadata[key]['title'][0][0].count("{")>0:
    tit=book.metadata[key]['title'][0][0].split("{")
    if len(' '.join(tit[0].split()))>4:
        title=' '.join(tit[0].split())
        series=tit[1].split("}")[0]
        #print title,"\n", series
    if len(series)>4:
        if "series" not in book.metadata[key].keys():
            book.metadata[key]['series']=[(series,{})]
        else:
            book.metadata[key]['series'].append((series,{}))

The Twelve 
Book Two of The Passage Trilogy


In [146]:
book.metadata[key]['series']

[('Book Two of The Passage Trilogy', {})]

In [97]:
#find the difference between a and b given that a is in b
#a=" [Lincoln Jeffery Deaver Rhyme 11.0] "
#b="Jeffery Deaver"
#output [lincoln rhyme 11.0]
def extract(a,b):
    if a.count(b)>0 and a.strip().lower() !=b.strip().lower():
        s=a[:a.index(b)]+a[a.index(b)+len(b):]
        return ' '.join(s.split()).title()
    elif b.count(a)>0 and b.strip().lower() !=a.strip().lower():
        s=b[:b.index(a)]+b[b.index(a)+len(a):]
        return ' '.join(s.split()).title()
    else:
        return ""

In [75]:
metadata["creator"][0][0]+"-"+metadata["title"][0][0]+".epub"

'Jeffery Deaver-The Skin Collector.epub'

In [26]:
def mvht(l):
    ddir="/media/some/sophie-hhd/books/thrill/"
    m,f=os.path.split(l)
    fn, fext = os.path.splitext(f)
    
    ff=ddir+fn+".epub"
    fe=m+"/"+fn+".epub"
    tmp="/tmp/"+fn+".epub"
    if not os.path.exists(ff) and not os.path.exists(fe): 
        try:
            !ebook-convert "$l" "$tmp" >/dev/null
            
            if os.path.exists(tmp):
                res=verifye(tmp)
                if res[1] == "OK":
                    !mv "$tmp" "$ff"
                else:
                    print "invalid ..",l,res
                    os.remove(tmp)
                #os.remove(l)
        except:
            print "faulty   ",l
    #if os.path.exists(l):
        os.remove(l)
    #print l

In [25]:
t0 = time.time()
p = Pool(8)
p.map(mvht, ls)
p.terminate()
p.close()
t1 = time.time()
total = t1-t0
print('time: ',total)

mv: cannot move ‘/tmp/Murder in the Cake - Chase, Kennedy.epub’ to ‘/media/some/sophie-hhd/books/thrill/Murder in the Cake - Chase, Kennedy.epub’: No such file or directory
mv: cannot move ‘/tmp/10 Vampire Mine - Sparks, Kerrelyn.epub’ to ‘/media/some/sophie-hhd/books/thrill/10 Vampire Mine - Sparks, Kerrelyn.epub’: No such file or directory
mv: cannot move ‘/tmp/Crazy Love You - Unger, Lisa.epub’ to ‘/media/some/sophie-hhd/books/thrill/Crazy Love You - Unger, Lisa.epub’: No such file or directory
mv: failed to access ‘/media/some/sophie-hhd/books/thrill/Book of Life, The - Harkness, Deborah.epub’: Permission denied
mv: failed to access ‘/media/some/sophie-hhd/books/thrill/In the Dark of Dreams (DS10) - Liu, Marjorie M_.epub’: Permission denied
Traceback (most recent call last):
  File "/usr/bin/ebook-convert", line 20, in <module>
    sys.exit(main())
  File "/usr/lib/calibre/calibre/ebooks/conversion/cli.py", line 360, in main
    plumber.run()
  File "/usr/lib/calibre/calibre/ebooks

In [7]:
def verifye(f):
    m,ff=os.path.split(f)
    try:
        if os.path.exists(f):  
            book=epub.read_epub(f)
            os.remove(f)
            epub.write_epub(f,book)
            return (f,"OK")
        else:
            return (f,"DOESNT EXIST")
            #print "file  ..." ,ff," ...... verified"
    except:
        #print "file corrupt ..." ,ff," ...... deleting"
        #os.remove(f)
        return (f,"NOT OK")

In [ ]:
4